In [1]:
# 취준생을 위한 서비스를 만들어야 합니다.

# 입력 : 사용자들이 입력 내용

# 직군(job_role)

# 숙련도(level)

# 출력 : json 형태

# 직군명(job_role)

# 숙련도 수준(level)

# 핵심 기술 역량(core_skills)

# 필수 도구 및 기술 스택(tools)

# 비기술 역량(soft_skills)

In [2]:
"""
취준생 역량 추천 서비스 (LangChain + LCEL + Output Parser)

입력: job_role, level
출력(JSON):
- job_role
- level
- core_skills
- tools
- soft_skills
"""

from dotenv import load_dotenv
import os

load_dotenv(override=True)
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

from pydantic import BaseModel, Field
from typing import List, Literal

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


# 1) 출력 스키마 정의 (Pydantic)
class SkillProfile(BaseModel):
    job_role: str = Field(description="직군명 (예: 데이터 분석가, 백엔드 개발자, AI 엔지니어 등)")
    level: str = Field(description="숙련도 수준 (예: 입문/주니어/미들/시니어 등)")
    core_skills: List[str] = Field(description="핵심 기술 역량 리스트 (5~10개)")
    tools: List[str] = Field(description="필수 도구 및 기술 스택 리스트 (5~12개)")
    soft_skills: List[str] = Field(description="비기술 역량 리스트 (4~8개)")


# 2) 파서 생성 (JSON 형태로 강제)
parser = JsonOutputParser(pydantic_object=SkillProfile)

# 3) 프롬프트 템플릿 (format_instructions 포함)
prompt_template = PromptTemplate(
    template="""
너는 취업 준비생을 돕는 커리어 코치다.
사용자가 입력한 직군(job_role)과 숙련도(level)에 맞춰,
현업에서 바로 통하는 역량을 간결하게 추천해라.

요구사항:
- 직군/숙련도에 맞는 "현실적인" 역량만 제시
- 리스트는 중복 없이, 너무 포괄적인 단어는 피하기
- core_skills는 기술 역량(개념/방법/실무 역량 중심)
- tools는 실제 현업에서 쓰는 도구/스택 중심
- soft_skills는 협업/커뮤니케이션/문제해결 역량 중심
- 출력은 반드시 JSON (아래 포맷)으로만 반환

{format_instructions}

입력:
- job_role: {job_role}
- level: {level}
""".strip(),
    input_variables=["job_role", "level"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4) LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",   # 필요 시 변경: "gpt-5-nano" 등
    api_key=OPENAI_API_KEY,
    temperature=0.3,
)

# 5) LCEL 체인 구성: prompt | llm | parser
chain = prompt_template | llm | parser


if __name__ == "__main__":
    # 예시 실행
    result = chain.invoke({"job_role": "데이터 분석가", "level": "주니어(1~3년)"})
    print(result)  # dict 형태로 나옴 (JSON 파싱 완료)


{'job_role': '데이터 분석가', 'level': '주니어', 'core_skills': ['기초 통계 분석', '데이터 시각화', 'SQL 쿼리 작성', '데이터 전처리', '기계 학습 기초'], 'tools': ['Python', 'Pandas', 'Tableau', 'Excel', 'Jupyter Notebook', 'R'], 'soft_skills': ['팀워크', '문제 해결 능력', '의사소통', '비판적 사고']}
